# Old CODE

In [ ]:
import argparse
import utils

files_group = {
    'datasets':['datasets/california_paper_eRCNN/I5-N-3/2015_train.csv'],
    'valid_set':['datasets/california_paper_eRCNN/I5-N-3/2016_val.csv'],
    'test_set':['datasets/california_paper_eRCNN/I5-N-3/2016_val.csv']
}
prediction_group = {
    'time_window':72,  # 'time window used to predict')
    'time_aggregation':1,  # 'steps aggregated for net input')
    'forecast_window':1,  # 'time window to be predicted')
    'forecast_aggregation':3  # 'steps aggregated in forecast')
}
training_group = {
    'train_set_size':70000,  # 'training set size')
    'valid_set_size':30000,  # 'validation set size')
    'valid_partitions':100,  # 'validation set partitions number')
    'test_partitions':100,  # 'test set partitions number')
    'batch_size':70,  # 'batch size for SGD')
    'learning_rate':1e-4,  # 'learning rate for SGD')
    'decay_rate':0.1,  # 'learning rate decay rate')
    'decay_steps':1000,  # 'learning rate decay steps')
    'gradient_clip':40.0,  # 'clip at this max norm of gradient')
    'max_steps':10000,  # 'max number of iterations for training')
    'save':'store_true',  # 'save the model every epoch')
    'ensemble':1  # 'Number of the model in the ensemble')
}
pickle_filename = utils.get_dataset_name(prediction_group['time_window'],
                                         prediction_group['time_aggregation'],
                                         prediction_group['forecast_window'],
                                         prediction_group['forecast_aggregation'],
                                         training_group['train_set_size'],
                                         training_group['valid_set_size'])

dataset = utils.get_dataset(pickle_filename, files_group, training_group, prediction_group)
train_set = dataset[0]
train_labels = dataset[1]
valid_set = dataset[2]
valid_labels = dataset[3]
valid_set2 = dataset[4]
valid_labels2 = dataset[5]
test_set = dataset[6]
test_labels = dataset[7]
mean = dataset[8]
stddev = dataset[9]
del dataset

print('Training set', train_set.shape, train_labels.shape)
print('Validation set', valid_set.shape, valid_labels.shape)
print('Test set', valid_set2.shape, valid_labels2.shape)

In [ ]:
wind_idx = 0
plt.title(f"Plot with args.forecast_aggregation = {prediction_group['forecast_aggregation']} ")
plt.scatter(train_labels[:, 13, wind_idx, 0], train_labels[:, 13, wind_idx, 2], s=40, facecolors='none', edgecolors='b')
plt.show()

In [ ]:
img = np.stack(
    [train_set[2,:,:,0]/np.max(train_set[2,:,:,0]),
    train_set[2,:,:,1]/np.max(train_set[2,:,:,1]),
    train_set[2,:,:,2]/np.max(train_set[2,:,:,2])], axis=2)
print(img.shape)
plt.imshow(img)
plt.show()

In [ ]:
cali_dataset_2015 = pd.read_csv("datasets/california_paper_eRCNN/I5-N-3/2015.csv")
cali_dataset_2016 = pd.read_csv("datasets/california_paper_eRCNN/I5-N-3/2016.csv")
cali_dataset_2016['timestep'] = cali_dataset_2016['timestep'] + cali_dataset_2015.tail(1)['timestep'].values + 1
cali_dataset = pd.concat([cali_dataset_2015, cali_dataset_2016])
cali_dataset.head()

In [ ]:
cali_dataset.describe()

In [ ]:
cali_avg_dataset = pd.DataFrame(columns=cali_dataset.columns)
cali_avg_dataset = cali_avg_dataset.append(
    [cali_dataset[cali_dataset["timestep"]==time].mean()
    for time in cali_dataset["timestep"].unique()],
    ignore_index=True)
cali_avg_dataset.head()

In [ ]:

cali_avg_dataset.describe()

In [ ]:
cali_avg_dataset.plot(
        x='flow',
        y='speed',
        style='o',
        title=f'Average Plot',
        xlim=[0,2500],
        ylim=[0,80],
        grid=True)

In [ ]:
cali_avg_dataset_2 = pd.DataFrame(columns=cali_dataset.columns)
cali_avg_dataset_2 = cali_avg_dataset.append(
    [cali_dataset[cali_dataset["timestep"]==time].median()
    for time in cali_dataset["timestep"].unique()],
    ignore_index=True)
cali_avg_dataset_2.head()

In [ ]:
cali_avg_dataset_2.plot.scatter(
        x='flow',
        y='speed',
        s=40,
        facecolors='r',
        edgecolors='none',
        title=f'Average Plot',
        xlim=[0,2500],
        ylim=[0,80],
        grid=True
)

In [ ]:
cali_avg_dataset.describe()

In [ ]:
for sensor in cali_dataset["road_section"].unique():
    cali_sensor_dataset = cali_dataset[cali_dataset["road_section"]==sensor]
    cali_sensor_dataset.plot(
        x='flow',
        y='speed',
        style='o',
        title=f'Sensor: {sensor}',
        xlim=[0,2500],
        ylim=[0,80],
        grid=True)
print(cali_sensor_dataset.head())